<h1> AWS Summit 2017 - Seoul: MXNet FNN Example

In [1]:
import mxnet as mx
import mxnet.ndarray as nd
import numpy as np
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)  # Config the logging
np.random.seed(777)

<h3> 1. Load the data </h3>

We use 8 features to predict whether the patient has diabetes or not. We will use 700 instances for training and 59 instances for testing.

In [2]:
xy = np.loadtxt(os.path.join('data-03-diabetes.csv'), delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

train_x_data, test_x_data = x_data[:700], x_data[700:]
train_y_data, test_y_data = y_data[:700], y_data[700:]
print(train_x_data.shape, test_x_data.shape, train_y_data.shape, test_y_data.shape)

# hyper-parameters
sample_num = x_data.shape[0]
dimension = x_data.shape[1]
batch_size = 32

(700, 8) (59, 8) (700, 1) (59, 1)


<h3> 2. Build the symbol </h3>

Next we will build the symbol, which is used to determine the data flow.

In [3]:
data = mx.sym.Variable("data")
target = mx.sym.Variable("target")
fc1 = mx.sym.FullyConnected(data=data, num_hidden=20, name='fc1')
fc1 = mx.sym.Activation(fc1, act_type="relu")
fc2 = mx.sym.FullyConnected(data=data, num_hidden=10, name='fc2')
fc2 = mx.sym.Activation(fc2, act_type="relu")
fc3 = mx.sym.FullyConnected(data=fc2, num_hidden=1, name='fc3')
pred = mx.sym.LogisticRegressionOutput(data=fc3, label=target)

<h3> 3. Construct the Module </h3>

We will construct the Module object based on the symbol. Module will be used for training and testing.

In [12]:
net = mx.mod.Module(symbol=pred,
                    data_names=['data'],
                    label_names=['target'],
                    context=mx.gpu(0))
net.bind(data_shapes = [mx.io.DataDesc(name='data', shape=(batch_size, dimension), layout='NC')],
         label_shapes= [mx.io.DataDesc(name='target', shape=(batch_size, 1), layout='NC')])
net.init_params(initializer=mx.init.Normal(sigma=0.01))
net.init_optimizer(optimizer='sgd', optimizer_params={'learning_rate': 0.2, 'momentum': 0.9})

<h3> 4. Training </h3>

We can fit the training set now.

In [13]:
# First constructing the training iterator and then fit the model
train_iter = mx.io.NDArrayIter(train_x_data, train_y_data, batch_size, shuffle=True, label_name='target')
metric = mx.metric.CustomMetric(feval=lambda labels, pred: ((pred > 0.5) == labels).mean(),
                                name="acc")
net.fit(train_data=train_iter, eval_metric=metric, num_epoch=15)

INFO:root:Epoch[0] Train-acc=0.657670


INFO:root:Epoch[0] Time cost=0.024


INFO:root:Epoch[1] Train-acc=0.657670


INFO:root:Epoch[1] Time cost=0.021


INFO:root:Epoch[2] Train-acc=0.723011


...

INFO:root:Epoch[12] Time cost=0.025


INFO:root:Epoch[13] Train-acc=0.768466


INFO:root:Epoch[13] Time cost=0.027


INFO:root:Epoch[14] Train-acc=0.774148


INFO:root:Epoch[14] Time cost=0.022


<h3> 5. Testing </h3>

Let's test the model on the test set.

In [14]:
test_iter = mx.io.NDArrayIter(test_x_data, None, batch_size, shuffle=False, label_name=None)

pred_class = (fc3 > 0)
test_net = mx.mod.Module(symbol=pred_class,
                         data_names=['data'],
                         label_names=None,
                         context=mx.gpu(0))
test_net.bind(data_shapes=[mx.io.DataDesc(name='data', shape=(batch_size, dimension), layout='NC')],
              label_shapes=None,
              for_training=False,
              shared_module=net)
out = test_net.predict(eval_data=test_iter)
print(out.asnumpy())

[[ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]


In [15]:
acc = np.equal(test_y_data, out.asnumpy()).mean()
print("Accuracy: %.4g" %acc)

Accuracy: 0.7797
